In [97]:
#!pip install python-Levenshtein

     |████████████████████████████████| 51kB 421kB/s eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144675 sha256=3de8cdf89a9dd90ffc4c82f36e3bccdc781e340fa2605ee873aecd549674cb9c
  Stored in directory: /home/chris/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [123]:
import pandas as pd
import numpy as np

In [153]:
df_transfer = pd.read_csv('../data/player_static.csv')
df_social = pd.read_csv('../data/social_media_followers.csv')
df_dream = pd.read_csv('../data/player_scores_parsed.csv')
df_noten = pd.read_csv('../data/player_noten.csv')


In [154]:
df_transfer_yb = df_transfer[df_transfer['team'] == 'BSC Young Boys']

In [155]:
def normalize_name(name):
    if name == 'David von Ballmoos':
        return 'David Von Ballmoos'
    elif name == 'Cedric Zesiger':
        return 'Cédric Zesiger'
    elif name == 'Mohamed Camara':
        return 'Mohamed Ali Camara'
    elif name == 'Jean Pierre Nsame':
        return 'Jean-Pierre Nsame'   
    elif name == 'Christopher Martins':
        return 'Christopher Martins Pereira'   
    else:
        return name

In [156]:
df_social['name'] = df_social['player'].apply(lambda x: normalize_name(x))
df_transfer_yb['name'] = df_social['name'].apply(lambda x: normalize_name(x))
df_dream['name'] = df_dream['player_name'].apply(lambda x: normalize_name(x))
df_social = df_social.drop('player', axis=1)
df_dream = df_dream.drop('player_name', axis=1)

In [157]:
df_social_transfer = pd.merge(df_social, df_transfer_yb,  how='left', left_on=['name'], right_on = ['name'])

In [158]:
df_dream_latest = df_dream[df_dream['season']==2100].groupby(['name']).agg({'score':'sum'}).reset_index()

In [159]:
df_merged = pd.merge(df_social_transfer, df_dream_latest,  how='left', left_on=['name'], right_on = ['name'])

In [160]:
ordered_columns = ['name', 
                  'team',
                  'transfermarktlink',
                  'picurl',
                  'position', 
                  'instagram',
                  'twitter',
                  'insta_followers',
                  'twitter_followers',
                  'marketvalue',
                  'score']
df_merged_ordered = df_merged[ordered_columns]
df_merged_ordered.columns = ['name', 
                  'team',
                  'url_trans',
                  'url_picture',
                  'position', 
                  'name_instagram',
                  'name_twitter',
                  'follower_instagram',
                  'follower_twitter',
                  'market_value',
                  'fantasy_score']

In [161]:
df_merged_ordered['market_value'].mean()

28831000.0

# 1/3 social 1/3 market value 1/3 fantasy score

In [162]:
# note transfer
df_merged_ordered['note'] = 0.0

df_noten['name'] = df_noten['name'].str.lower()

df_merged_ordered['note'] = df_merged_ordered['name'].apply(lambda x: float(df_noten.loc[df_noten['name']==x.lower()]['note'].iloc[0]) if x.lower() in df_noten['name'].values else 0.0)

/home/chris/Varia/YBHack19/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/chris/Varia/YBHack19/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [163]:
df_merged_ordered['follower_instagram_norm'] = df_merged_ordered['follower_instagram'].apply(lambda x: (x-df_merged_ordered['follower_instagram'].min())/(df_merged_ordered['follower_instagram'].max()-df_merged_ordered['follower_instagram'].min()))

df_merged_ordered['follower_twitter_norm'] = df_merged_ordered['follower_twitter'].apply(lambda x: (x-df_merged_ordered['follower_twitter'].min())/(df_merged_ordered['follower_twitter'].max()-df_merged_ordered['follower_twitter'].min()))
df_merged_ordered['market_value_norm'] = df_merged_ordered['market_value'].apply(lambda x: (x-df_merged_ordered['market_value'].min())/(df_merged_ordered['market_value'].max()-df_merged_ordered['market_value'].min()))
df_merged_ordered['fantasy_score_norm'] = df_merged_ordered['fantasy_score'].apply(lambda x: (x-df_merged_ordered['fantasy_score'].min())/(df_merged_ordered['fantasy_score'].max()-df_merged_ordered['fantasy_score'].min()))
df_merged_ordered['note_norm'] = df_merged_ordered['note'].apply(lambda x: (x-df_merged_ordered['note'].min())/(df_merged_ordered['note'].max()-df_merged_ordered['note'].min()))

/home/chris/Varia/YBHack19/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/chris/Varia/YBHack19/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [164]:
df_merged_ordered['fame'] = 0.0
df_merged_ordered['fame'] = df_merged_ordered.apply(lambda x: round(100*(0.25 * df_merged_ordered['follower_instagram_norm']+ 0.25 * df_merged_ordered['follower_twitter_norm'] + 0.5 * df_merged_ordered['note_norm']  ),1),axis=1)

## Save to CSV

In [165]:
df_merged_ordered.to_csv( '../data/df_static_merged.csv', index=False)

In [166]:
df_merged_ordered

name            team  \
0                  Marco Wölfli  BSC Young Boys   
1            David Von Ballmoos  BSC Young Boys   
2                 Dario Marzino  BSC Young Boys   
3             Frederik Sörensen  BSC Young Boys   
4            Mohamed Ali Camara  BSC Young Boys   
5                Cédric Zesiger  BSC Young Boys   
6                 Nicolas Bürgy  BSC Young Boys   
7                Ulisses Garcia  BSC Young Boys   
8                   Saidy Janko  BSC Young Boys   
9           Fabian Lustenberger  BSC Young Boys   
10               Jordan Lotomba  BSC Young Boys   
11             Esteban Petignat  BSC Young Boys   
12             Marvin Spielmann  BSC Young Boys   
13               Vincent Sierro  BSC Young Boys   
14            Miralem Sulejmani  BSC Young Boys   
15             Gianluca Gaudino  BSC Young Boys   
16       Nicolas Moumi Ngamaleu  BSC Young Boys   
17          Christian Fassnacht  BSC Young Boys   
18             Michel Aebischer  BSC Young Boys   
19                Sandro Lauper  BSC Young Boys   
20  Christopher Martins Pereira  BSC Young Boys   
21                 Roger Assalé  BSC Young Boys   
22            Jean-Pierre Nsame  BSC Young Boys   
23               Felix Mambimbi  BSC Young Boys   
24             Guillaume Hoarau  BSC Young Boys   

                                            url_trans  \
0           /david-von-ballmoos/profil/spieler/203124   
1                   /marco-wolfli/profil/spieler/4860   
2                /dario-marzino/profil/spieler/254968   
3               /mohamed-camara/profil/spieler/426723   
4           /fabian-lustenberger/profil/spieler/42203   
5             /frederik-sorensen/profil/spieler/91716   
6               /cedric-zesiger/profil/spieler/382478   
7                /nicolas-burgy/profil/spieler/203895   
8               /ulisses-garcia/profil/spieler/192616   
9               /jordan-lotomba/profil/spieler/313094   
10                 /saidy-janko/profil/spieler/196435   
11  /christopher-martins-pereira/profil/spieler/24...   
12            /michel-aebischer/profil/spieler/237658   
13              /vincent-sierro/profil/spieler/280387   
14               /sandro-lauper/profil/spieler/254965   
15            /gianluca-gaudino/profil/spieler/195051   
16            /esteban-petignat/profil/spieler/479480   
17         /christian-fassnacht/profil/spieler/250490   
18            /marvin-spielmann/profil/spieler/192653   
19            /miralem-sulejmani/profil/spieler/36080   
20              /felix-mambimbi/profil/spieler/507496   
21      /nicolas-moumi-ngamaleu/profil/spieler/266768   
22                /roger-assale/profil/spieler/263183   
23           /jean-pierre-nsame/profil/spieler/225055   
24             /guillaume-hoarau/profil/spieler/23934   

                                          url_picture           position  \
0   https://tmssl.akamaized.net//images/portrait/h...         Goalkeeper   
1   https://tmssl.akamaized.net//images/portrait/h...         Goalkeeper   
2   https://tmssl.akamaized.net//images/portrait/h...         Goalkeeper   
3   https://tmssl.akamaized.net//images/portrait/h...        Centre-Back   
4   https://tmssl.akamaized.net//images/portrait/h...        Centre-Back   
5   https://tmssl.akamaized.net//images/portrait/h...        Centre-Back   
6   https://tmssl.akamaized.net//images/portrait/h...        Centre-Back   
7   https://tmssl.akamaized.net//images/portrait/h...        Centre-Back   
8   https://tmssl.akamaized.net//images/portrait/h...          Left-Back   
9   https://tmssl.akamaized.net//images/portrait/h...         Right-Back   
10  https://tmssl.akamaized.net//images/portrait/h...         Right-Back   
11  https://tmssl.akamaized.net//images/portrait/h...  DefensiveMidfield   
12  https://tmssl.akamaized.net//images/portrait/h...    CentralMidfield   
13  https://tmssl.akamaized.net//images/portrait/h...    CentralMidfield   
14  https://tmssl.akamaized.net//images/portrait/h...    CentralMidfield   
1